In [ ]:
import numpy as np
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
def int2bin(i):
    b = bin(i)[2:].zfill(10)
    return np.array([int(i) for i in b])

def bin2in(s):
    return int(s, 2)

In [ ]:
reset_graph()

n_steps   = 10
n_inputs  = 2
n_neurons = 3
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

sm = tf.nn.sigmoid(outputs)

init = tf.global_variables_initializer()

In [ ]:
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(sm-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [ ]:
def gen_data():
    max_val = 100
    A = np.random.randint(max_val)
    B = np.random.randint(max_val)
    C = A+B
    Ab = int2bin(A)
    Bb = int2bin(B)
    Cb = int2bin(C)
        
    X_batch = None
    X_batch = np.vstack((Ab, Bb)).T
    X_batch = X_batch[np.newaxis, :]
    y_batch = Cb[np.newaxis, :, np.newaxis]
    
    return X_batch, y_batch

In [ ]:
X_batch, y_batch = gen_data()
print(X_batch[0].T[0])
print(X_batch[0].T[1])
print("sum")
print(y_batch[0].T[0])

In [ ]:
n_iterations = 60000
# batch_size   = 50

saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = gen_data()
        
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

        if iteration % 5000 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            
            print(iteration, "\tMSE:", mse)
    saver.save(sess, "./model")

In [ ]:
def reformat(tens):
    return tens[0].T[0]

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./model")
    X_batch, y_batch = gen_data()
    res = sm.eval(feed_dict={X: X_batch, y: y_batch})
#     res = states.eval(feed_dict={X: X_batch, y: y_batch})
#     print(res)
    res[res>0.5] = 1
    res[res<=0.5] = 0
    print(reformat(res))
    print(reformat(y_batch))